<a href="https://colab.research.google.com/github/VIVEK-JADHAV/MovieRecommendation/blob/main/ColdStartFormulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Importing libraries

import pandas as pd
import numpy as np
import os
from ast import literal_eval
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.sparse import csr_matrix,coo_matrix

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Loading the data

os.mkdir('/content/data')
os.chdir('/content/data')
print(os.getcwd())

!unzip '/content/drive/MyDrive/CaseStudies/RecommendationSystem/archive.zip'

/content/data
Archive:  /content/drive/MyDrive/CaseStudies/RecommendationSystem/archive.zip
  inflating: credits.csv             
  inflating: keywords.csv            
  inflating: links.csv               
  inflating: links_small.csv         
  inflating: movies_metadata.csv     
  inflating: ratings.csv             
  inflating: ratings_small.csv       


### Cold Start Problem


In [ ]:
movies_df=pd.read_csv('/content/data/movies_metadata.csv')
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.9469,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.0155,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,11.7129,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",3.85949,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,8.38752,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

The required columns are genres,vote_average,vote_count,title

In [ ]:
#Keeping only the required columns

movies=movies_df[['title','genres','vote_average','vote_count','original_language','id']]
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   title              45460 non-null  object 
 1   genres             45466 non-null  object 
 2   vote_average       45460 non-null  float64
 3   vote_count         45460 non-null  float64
 4   original_language  45455 non-null  object 
 5   id                 45466 non-null  object 
dtypes: float64(2), object(4)
memory usage: 2.1+ MB


In [ ]:
#Null values
movies.isnull().sum()

title                 6
genres                0
vote_average          6
vote_count            6
original_language    11
id                    0
dtype: int64

In [ ]:
#Rows with null values
movies[movies.isnull().any(axis=1)==True]

,title,genres,vote_average,vote_count,original_language,id
19574,Shadowing the Third Man,"[{'id': 99, 'name': 'Documentary'}]",0.0,0.0,NaN,283101
19729,NaN,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,NaN,en,82663
19730,NaN,"[{'name': 'Carousel Productions', 'id': 11176}...",NaN,NaN,104.0,1997-08-20
21602,Unfinished Sky,"[{'id': 10749, 'name': 'Romance'}, {'id': 18, ...",6.4,8.0,NaN,103902
22832,13 Fighting Men,"[{'id': 10752, 'name': 'War'}, {'id': 37, 'nam...",0.0,0.0,NaN,359195
29502,NaN,"[{'id': 16, 'name': 'Animation'}, {'id': 878, ...",NaN,NaN,ja,122662
29503,NaN,"[{'name': 'Aniplex', 'id': 2883}, {'name': 'Go...",NaN,NaN,68.0,2012-09-29
32141,Lambchops,"[{'id': 35, 'name': 'Comedy'}]",6.5,2.0,NaN,147050
35586,NaN,"[{'id': 10770, 'name': 'TV Movie'}, {'id': 28,...",NaN,NaN,en,249260
35587,NaN,"[{'name': 'Odyssey Media', 'id': 17161}, {'nam...",NaN,NaN,82.0,2014-01-01


In [ ]:
#Dropping Nan values
movies.dropna(inplace=True)
movies.isnull().sum()

title                0
genres               0
vote_average         0
vote_count           0
original_language    0
id                   0
dtype: int64

In [ ]:
#Transforming genre column with first occuring value
# Empty generes are replaced by Drama as it is most occuring value
def transformGenere(genre):
  lst=[]
  for i in genre:    
    if i['name']:
      lst.append(i['name'])
  return lst if len(lst)>0 else ["Drama"]
movies['genres']=movies['genres'].apply(literal_eval).apply(transformGenere)   #literal_eval removes quotes around list
movies.head()


,title,genres,vote_average,vote_count,original_language,id
0,Toy Story,"[Animation, Comedy, Family]",7.7,5415.0,en,862
1,Jumanji,"[Adventure, Fantasy, Family]",6.9,2413.0,en,8844
2,Grumpier Old Men,"[Romance, Comedy]",6.5,92.0,en,15602
3,Waiting to Exhale,"[Comedy, Drama, Romance]",6.1,34.0,en,31357
4,Father of the Bride Part II,[Comedy],5.7,173.0,en,11862


In [ ]:
#List of all the genres and languages
genres=set()
language=set()
for i,rows in movies.iterrows():
  for j in rows['genres']:
    genres.add(j)
  # print(rows['genres'])
  # break
  
  language.add(rows['original_language'])

print(genres)
print(language)

{'Documentary', 'Family', 'History', 'Fantasy', 'Horror', 'Science Fiction', 'Foreign', 'Drama', 'Thriller', 'Mystery', 'Crime', 'Western', 'Action', 'War', 'TV Movie', 'Comedy', 'Adventure', 'Romance', 'Animation', 'Music'}
{'tl', 'nb', 'lv', 'fy', 'bn', 'fr', 'cn', 'th', 'hu', 'mn', 'ku', 'si', 'bs', 'en', 'tr', 'ay', 'sr', 'eo', 'am', 'kn', 'ro', 'nl', 'uk', 'tg', 'sh', 'fa', 'ml', 'fi', 'ur', 'no', 'lb', 'la', 'el', 'cs', 'zh', 'qu', 'hr', 'ca', 'sl', 'mr', 'af', 'zu', 'pl', 'ky', 'xx', 'is', 'de', 'wo', 'sm', 'ja', 'bg', 'rw', 'bm', 'jv', 'kk', 'sk', 'cy', 'eu', 'ka', 'hy', 'ab', 'sv', 'ko', 'id', 'ps', 'ms', 'vi', 'lo', 'it', 'pt', 'gl', 'sq', 'ru', 'es', 'bo', 'he', 'pa', 'mk', 'te', 'ne', 'hi', 'lt', 'ar', 'da', 'et', 'ta', 'uz', 'mt', 'iu'}


In [ ]:
import re

def mapCodeToLanguage(row):
  d={}
  for i in row:
    key=i['iso_639_1']
    if key not in d:
      d[key]=i['name']
  return d


spoken_language=movies_df['spoken_languages'].dropna().apply(literal_eval).apply(mapCodeToLanguage)

uniqueLanguage= set()
for i in spoken_language:
  for k,v in i.items():
    if v and re.match(r"[a-zA-Z]+",v):
      uniqueLanguage.add((k,v))

print(uniqueLanguage)

{('tr', 'Türkçe'), ('gl', 'Galego'), ('ro', 'Română'), ('id', 'Bahasa indonesia'), ('pt', 'Português'), ('sr', 'Srpski'), ('bm', 'Bamanankan'), ('hu', 'Magyar'), ('la', 'Latin'), ('sq', 'shqip'), ('et', 'Eesti'), ('lt', 'Lietuvi\x9akai'), ('sv', 'svenska'), ('it', 'Italiano'), ('nl', 'Nederlands'), ('uz', 'ozbek'), ('ca', 'Català'), ('ms', 'Bahasa melayu'), ('ru', 'Pусский'), ('da', 'Dansk'), ('vi', 'Tiếng Việt'), ('rw', 'Kinyarwanda'), ('fr', 'Français'), ('mt', 'Malti'), ('pl', 'Polski'), ('wo', 'Wolof'), ('eo', 'Esperanto'), ('de', 'Deutsch'), ('af', 'Afrikaans'), ('so', 'Somali'), ('sl', 'Slovenščina'), ('az', 'Azərbaycan'), ('bs', 'Bosanski'), ('nb', 'Bokmål'), ('ff', 'Fulfulde'), ('cy', 'Cymraeg'), ('fi', 'suomi'), ('xx', 'No Language'), ('zu', 'isiZulu'), ('hr', 'Hrvatski'), ('ga', 'Gaeilge'), ('eu', 'euskera'), ('en', 'English'), ('no', 'Norsk'), ('ha', 'Hausa'), ('es', 'Español'), ('lv', 'Latviešu'), ('sk', 'Slovenčina'), ('sw', 'Kiswahili')}


Using the Weighted Rating formula of IMDB,

Weighted Rating (WR) =  v.R/(v+m)+m.C(v+m)

where:
* v is the number of votes for the movie
* m is the minimum votes required to be listed in the chart
* R is the average rating of the movie
* C is the mean vote across the whole report

In [ ]:
#Setting the m value as 85th percentile of vote_count and c as mean of vote_average
m=np.percentile(movies['vote_count'],85)
print(m)

c=np.mean(movies['vote_average'])
print(c)

82.0
5.618803494025977


In [ ]:
def filterByGenre(genre,t):
  ''' Returns true if the given genre is in genre list'''
  if t in genre:
    return True
  else:
    return False

def computeWtScore(row,m,c):
  '''Compute the weighted score'''
  v=row['vote_count']
  r=row['vote_average']
  result=v*r/(v+m)+m*c/(v+m)
  return result


def coldstart(movies,genre,lang,m,c):
  temp=movies[(movies['genres'].apply(lambda x: filterByGenre(x,genre))) & (movies['original_language']==lang)]  
  temp['WtScore']=temp.apply(lambda row: computeWtScore(row,m,c),axis=1)
  temp.sort_values('WtScore',inplace=True,ascending=False)
  return temp 



In [ ]:
coldstart(movies,'Action','hi',m,c).head(10)

,title,genres,vote_average,vote_count,original_language,id,WtScore
12418,Jodhaa Akbar,"[Action, Adventure, Drama, History, Romance]",7.6,74.0,hi,14073,6.558602
20688,Don : The Chase Begins Again,"[Action, Drama, Thriller]",7.3,68.0,hi,17501,6.380946
21256,Bhaag Milkha Bhaag,"[History, Action, Drama]",7.3,62.0,hi,206324,6.342652
24565,Haider,"[Action, Drama, Crime, Romance]",7.2,65.0,hi,280795,6.317972
36014,Airlift,"[Thriller, Action, Drama, History]",7.3,58.0,hi,375290,6.315299
19637,Gangs of Wasseypur,"[Action, Thriller, Crime]",7.0,83.0,hi,117691,6.313587
22764,Don 2: The King is Back,"[Action, Thriller]",7.1,65.0,hi,41109,6.273754
16451,Ghajini,"[Thriller, Action, Drama, Mystery]",6.9,84.0,hi,14070,6.267120
39495,Sultan,"[Drama, Action]",7.0,71.0,hi,376812,6.259751
23333,A Wednesday!,"[Action, Drama]",7.0,61.0,hi,19703,6.207985
